In [ ]:
'''
装饰器本质上就是一个函数，它可以让其他函数在不需要做任何代码变动的前提下，增加额外的功能，
装饰器的返回值是一个函数对象（函数的指针）
'''

In [4]:
# _*_ 1.函数即对象 _*_ #

# 性质1：可以赋值于其他变量
def foo():
    print('func: foo')

ff = foo
ff()
foo()
print(id(ff), id(foo)) # 指向同一个地址

# 性质2：可以作为参数传递和返回值
def bar(func):
    print('func: bar')
    func()
    
def boo():
    print('func: boo')
    
bb = bar(boo)

# 注意下面两个的区别
bb
print(bb)

func: foo
func: foo
1490472107024 1490472107024
func: bar
func: boo
None


In [18]:
# _*_ 2.函数嵌套与闭包 _*_ #
# 函数嵌套：即在函数内部，还有函数定义
# 闭包：一个函数中的嵌套函数，引用 enclosing 域内的变量，则该嵌套函数即为闭包

# eg.1
def add_outer(outer):
    outer = 0
    def add_inner(inner):
        outer = 1
        return outer + inner
    return add_inner

a1 = add_outer(12)
a2 = add_outer(13)
print(a1(100))
print(a2(100))

'''
outer既是外部函数add_outer的参数，又是嵌套函数add_inner的变量，于是，可以看做函数add_inner的一个配置configuration
配置信息不同，add_inner 的功能也就不同，这就是闭包
'''

# eg.2
def hellocounter(name):
    count = 0
    def counter():
        nonlocal count 
        count += 1
        print('Hello ',name,'! The ',str(count),' times.')
    return counter

hello = hellocounter('Alex')
hello()
hello()
hello()
print(hello())

# eg.3 装饰器：对函数(参数，返回值等)进行加工处理，生成一个功能增强版的函数！
def make_bold(n):
    def wrapped():
        return n()
    return wrapped

def make_italic(n):
    def wrapped():
        return n()
    return wrapped

@make_bold
@make_italic
def hello():
    return 'hello world!'

print(hello())

101
101
Hello  Alex ! The  1  times.
Hello  Alex ! The  2  times.
Hello  Alex ! The  3  times.
Hello  Alex ! The  4  times.
None
hello world!


In [4]:
# 闭包用途1：闭包执行完成后，仍然能够保持住当前的运行环境，即：本次执行结果可以当做下次执行的初始值
origin = [0,0]
def create(pos=origin):
    def player(dis,step):
        pos[0] = pos[0] + dis[0] * step
        pos[1] = pos[1] + dis[1] * step
        return pos
    print('pos:',pos)
    return player

player = create()
print(player([1,0], 10))
print(player([0,1], 20))
print(player([-1,0], 10))

# 闭包用途2：闭包可以根据外部作用域的局部变量，达到配置(configuration)功能
def make_filter(keep):
    def filter(filename):
        with open(filename,'r') as f:
            data = [i for i in f.readlines() if keep in i]
            ''' # 等价于以下 code
            for line in f.readlines():
                if keep in line.strip().split(' '):
                    data.append(line)
                else:
                    continue
            '''
        return data
    return filter

mf = make_filter('pass')
mf('pass.txt')

mf = make_filter('who')
mf('pass.txt')

pos: [0, 0]
[10, 0]
[10, 20]
[0, 20]


['who  are you']

In [5]:
# _*_ 带参数的被包装函数 _*_ #
import time
def show_time(func):
    def wrapper(*args, **kwargs):  # wrapper()可以使用参数 func ，因为这是一个闭包
        start = time.time()
        func(*args, **kwargs)
        print(time.time()-start)
    return wrapper

@show_time # 等价于 foo = show_time(foo) 
def foo(a, b):
    print(a + b)
    time.sleep(0.5)
    
@show_time # 等价于 bar = show_time(bar)
def bar(a, b):
    print(a * b)
    time.sleep(0.2)
    
@show_time
def add(*args, **kwargs):
    time.sleep(0.1)
    sum = 0
    for i in args:
        sum += i
    print(sum)

foo(2, 3)
print('*' * 20)
bar(2, 3)
print('*' * 20)
add(2,5,6,9,4) # *args 不是 list

5
0.5095934867858887
********************
6
0.20775103569030762
********************
26
0.10629534721374512


In [6]:
# _*_ 带参数的包装函数 _*_ #
'''
1. log是带参数的装饰器，并返回一个装饰器(含参数的闭包函数)
2. @log(1) : add = show_time(add)(flag=1)
'''
def log(flag=0):
    def show_time(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            func(*args, **kwargs)
            print('spend time: ',time.time()-start)
            
            if flag:
                print('This event will login!')
        return wrapper
    return show_time

@log(1)
def foo():
    time.sleep(0.3)
    print('foo function')
    
foo()

foo function
spend time:  0.30928468704223633
This event will login!


In [7]:
# _*_ 多层装饰器：注意装饰顺序 _*_ #
def makebold(func):
    def wrapper():
        return '<b>' + func() + '</b>'
    return wrapper

def makeitealic(func):
    def wrapper():
        return '<i>' + func() + '</i>'
    return wrapper

@makebold    # hello = makebold(hello)    -> wrapper2
@makeitealic # hello = makeitealic(hello) -> wrapper1
def hello():
    return 'hello world!'
    
hello()

'<b><i>hello world!</i></b>'

In [13]:
# _*_ 类装饰器：封装与继承，当用 @ 形式将装饰器附加到函数上时，会调用class.__call__方法 _*_ #
class Timer(object):
    def __init__(self, func):
        self._func = func # self._func class的私有函数
        self._name = 'timer'
        
    def __call__(self):
        start = time.time()
        self._func()
        print('spend time:',time.time()-start)
        
    def __name__(self):
        return self._name
        
@Timer # bar = Timer(bar)
def bar():
    time.sleep(1)
    print('bar func')
    print(bar.__name__) # 使用装饰器后，函数的元信息变成了装饰器的元信息
    
bar()


def foo():
    time.sleep(0.5)
    print("foo func")
    print(foo.__name__)
    
foo()


bar func
<bound method Timer.__name__ of <__main__.Timer object at 0x000001844F219100>>
spend time: 1.0003278255462646
foo func
foo


In [18]:
# _*_ functools.wraps(自身也是一个装饰器) _*_ #
# 将原函数的元信息拷贝到装饰器中，使得装饰器函数与原函数的元信息一致
from functools import wraps

def logger(func):
    @wraps(func) # 必须将function传递给wraps
    def wrapper(*args, **kwargs):
        print(func," metadata ",func.__name__)
        return func(*args, **kwargs)        
    return wrapper

@logger
def foo(name):
    print('func name:',name)
    
foo('alex')

@logger
def bar():
    pass

bar()

<function foo at 0x000001844FE8C8B0>  metadata  foo
func name: alex
<function bar at 0x000001844F560AF0>  metadata  bar
